In [10]:
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import time
import re

API_KEY = '3HutAvbl1TRuIXG0mjgNjTGPuLy_ZhoON7UEYzJqG6o2D6gPkIVZr1RT4x3TBpR3iXtJ9lBnis50H7MXRp0uSisapP-8Tup98qEipjCL6AjzLv2jHM6UhMXZN8IBYnYx'
yelp_api = YelpAPI(API_KEY)

ModuleNotFoundError: No module named 'yelpapi'

In [11]:
# STEP 1: create list of neighborhoods for each midwest city
cities = ['Chicago, Illinois',
 'Columbus, Ohio', 
 'Indianapolis, Indiana',
 'Detroit, Michigan',
 'Milwaukee, Wisconsin',
 'Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota']    # changed St. Paul to Saint Paul

# scraping wikipedia page
# https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
page = ''
while page == '':
    try:
        page = requests.get('https://en.wikipedia.org/wiki/Lists_of_neighborhoods_by_city#United_States')
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue
        
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
items = content.find_all('li')

In [12]:
# STEP 1.1: get directories (what comes after wikipedia base url) for all cities
# creating dict where keys are cities and values are directories
city_directories = {}

# iterate through list of cities on wikipedia page - if they match our cities, get its directory
for item in items:
    wiki_city_cleaned = item.text.strip('*').strip()
    if wiki_city_cleaned == 'Neighborhoods of Chicago':   # hard-coded
        city_directories[cities[0]] = item.a['href']
    for city in cities:
        if wiki_city_cleaned == city.split(',')[0] or wiki_city_cleaned == city: 
            city_directories[city] = item.a['href']

city_directories['Kansas City, Missouri'] = '/wiki/Neighborhoods_of_Kansas_City,_Missouri'   # hard-coded
city_directories

{'Chicago, Illinois': '/wiki/Neighborhoods_of_Chicago',
 'Columbus, Ohio': '/wiki/Neighborhoods_in_Columbus,_Ohio',
 'Detroit, Michigan': '/wiki/Neighborhoods_in_Detroit',
 'Indianapolis, Indiana': '/wiki/Neighborhoods_of_Indianapolis',
 'Milwaukee, Wisconsin': '/wiki/Neighborhoods_of_Milwaukee',
 'Minneapolis, Minnesota': '/wiki/Neighborhoods_of_Minneapolis',
 'Saint Paul, Minnesota': '/wiki/Neighborhoods_of_Saint_Paul',
 'St. Louis, Missouri': '/wiki/Neighborhoods_of_St._Louis',
 'Kansas City, Missouri': '/wiki/Neighborhoods_of_Kansas_City,_Missouri'}

In [24]:
# STEP 1.2: creating functions for scraping wikipedia page & cleaning

base_url = 'https://en.wikipedia.org'

# uses pandas read_html to scrape neighborhoods from table on each wiki page
def read_wiki(city):
    directory = city_directories.get(city)
    wiki = pd.read_html(base_url + directory)
    return wiki

# uses bs4 to scrape neighborhoods based on html tags
def soup(city):
    directory = city_directories.get(city)
    page = requests.get(base_url + directory)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return content

# cleans where neighborhoods have multiple names with '/'
# NOTE: still in progress
def cleaned(lst):
    new_lst = []
    for neighborhood in lst:
        if re.search(r"^[\w\s]+\s?[(\/\[]", neighborhood):
            if "/" in neighborhood:
                towns = neighborhood.split("/")
                stripped_towns = [i.strip() for i in towns]
                new_lst.extend(stripped_towns)
            elif "[" in neighborhood:
                matches = re.findall(r"^([\w\s]+)\s?[(\/\[]", neighborhood)
                stripped_matches = [i.strip() for i in matches]
                if matches:
                    new_lst.extend(stripped_matches)
            else:
                new_neighborhood = neighborhood.replace("(", "").replace(")", "").strip()
                new_lst.append(new_neighborhood)
        else:
            new_lst.append(neighborhood.strip())
    return new_lst
    
    
    
#     for neighborhood in lst:
#         if '/' in neighborhood or ' / ' in neighborhood:
#             lst.remove(neighborhood)
#             aliases = neighborhood.split('/')
#             for item in aliases:
#                 lst.append(item.strip())
#         if '[' in neighborhood:
#             lst.remove(neighborhood)
#             split = neighborhood.split('[')
#             lst.append(split[0].strip())
    return lst

In [27]:
# STEP 1.3: get neighborhoods for chicago, indianapolis, detroit, st. louis
# creating dict where key is city and value is list of neighborhoods
neighborhoods = {}

# chicago neighborhoods
wiki_chicago = read_wiki('Chicago, Illinois')
chicago_neighborhoods = wiki_chicago[0]
chicago_neighborhoods = list(chicago_neighborhoods['Neighborhood'])
neighborhoods['Chicago, Illinois'] = chicago_neighborhoods

# indianapolis neighborhoods
wiki_ind = read_wiki('Indianapolis, Indiana')
ind_neighborhoods = wiki_ind[2]
indianapolis_neighborhoods = list(ind_neighborhoods[ind_neighborhoods.columns[0]])
neighborhoods['Indianapolis, Indiana'] = indianapolis_neighborhoods

# st. louis neighborhoods
wiki_stl = read_wiki('St. Louis, Missouri')
stl_neighborhoods = wiki_stl[1]
stl_neighborhoods = list(stl_neighborhoods['Neighborhood'])
neighborhoods['St. Louis, Missouri'] = stl_neighborhoods

# detroit neighborhoods
wiki_detroit =  read_wiki('Detroit, Michigan')
detroit_neighborhoods = []
for page in wiki_detroit:
    if 'Name' in page.columns:
        detroit_neighborhoods.extend(list(page['Name']))
        
# cleaning detroit neighborhoods
detroit_neighborhoods = cleaned(detroit_neighborhoods)
for n in detroit_neighborhoods:
    if '/' in n:
        detroit_neighborhoods.remove(n)
        names = n.split('/')
        for name in names:
            detroit_neighborhoods.append(name)
neighborhoods['Detroit, Michigan'] = detroit_neighborhoods

print(detroit_neighborhoods)

['Bricktown Historic District', 'Broadway Avenue Historic District', 'Campus Martius Park', 'Capitol Park Historic District', 'Detroit Financial District', 'Grand Circus Park Historic District', 'Greektown', 'Jefferson Avenue', 'Lower Woodward Avenue Historic District', 'Monroe Avenue Commercial Buildings', 'Park Avenue Historic District', 'Randolph Street Commercial Buildings Historic District', 'Washington Boulevard Historic District', 'Brush Park', 'Woodward East', 'Cass Corridor', 'Cass Park Historic District', 'Cass-Davenport Historic District', 'Cultural Center Historic District', 'East Ferry Avenue Historic District', 'Midtown Woodward Historic District', 'Sugar Hill Historic District', 'University-Cultural Center', 'Warren-Prentis Historic District', 'Wayne State University', 'West Canfield Historic District', 'Willis-Selden Historic District', 'Woodbridge', 'Arden Park-East Boston Historic District', 'Atkinson Avenue Historic District', 'Boston-Edison Historic District', 'New 

In [15]:
# STEP 1.4: get neighborhoods for columbus, milwaukee, kansas city, minneapolis, saint paul 
# NOTE: need fix repetition of code later for efficiency

# columbus neighborhoods
columbus_directory = city_directories.get('Columbus, Ohio')
page = requests.get(base_url + columbus_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
columbus_neighborhoods = [item.text.replace('[edit]', '') for item in content.find_all(['h2', 'h3', 'h4'])][2:-3]
neighborhoods['Columbus, Ohio'] = columbus_neighborhoods

# milwaukee neighborhoods
mlwk_directory = city_directories.get('Milwaukee, Wisconsin')
page = requests.get(base_url + mlwk_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
mlwk_neighborhoods = [item.text.replace('[edit]', '').strip('"') for item in content.find_all('h3')]
neighborhoods['Milwaukee, Wisconsin'] = mlwk_neighborhoods

# kansas neighborhoods
kansas_directory = city_directories.get('Kansas City, Missouri')
page = requests.get(base_url + kansas_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
li_list = [item.text for item in content.find_all('li')]
kansas_neighborhoods = li_list[li_list.index('CBD-Downtown'):li_list.index('Wornall Homestead') + 1]
neighborhoods['Kansas City, Missouri'] = kansas_neighborhoods

# minneapolis neightborhoods
minneapolis_directory = city_directories.get('Minneapolis, Minnesota')
page = requests.get(base_url + minneapolis_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3 = [item.text.replace('[edit]', '').strip('"') for item in content.find_all(['h3'])]
li = [item.text for item in content.find_all(['li']) if item.text[0].isalpha()]
li = li[0:li.index('University') + 1]
b = [item.text for item in content.find_all('b') if item.text[0].isalpha() and item.text not in li]
minneapolis_neighborhoods = h3 + li + b
neighborhoods['Minneapolis, Minnesota'] = minneapolis_neighborhoods

# saint paul neighborhoods
stpaul_directory = city_directories.get('Saint Paul, Minnesota')
page = requests.get(base_url + stpaul_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3_list = [item.text.strip().split('- ')[1].replace('[edit]', '') for item in content.find_all('h3')]
stpaul_neighborhoods = []
for h3 in h3_list:
    names = h3.split(',')
    for name in names:
        stpaul_neighborhoods.append(name.strip())
neighborhoods['Saint Paul, Minnesota'] = stpaul_neighborhoods

In [16]:
# step 2: use yelp API to get restaurant data for each neighborhood
# starting with detroit
# how many restaurants do we want for each neighborhood? aka what limit

for n in neighborhoods['Detroit, Michigan']:
    try:
        print(yelp_api.search_query(categories='restaurants', location=n + ', Detroit', limit=1))
    except YelpAPI.YelpAPIError as e:
        print('Error')

KeyError: 'Detroit, Michigan'

In [ ]:
# step 3: write results into csv file (or create pandas dataframe?) for each neighborhood